Let's see if we can run the astrometry metrics in a vectorized manner, so we can easily visualize the magnitude variation of things

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db
import lsst.sims.maf.plots as plots

In [2]:
# Let's make a wrapper metric for vectorizing things on magnitude
class MagWrapper(metrics.vectorMetrics.VectorMetric):
    def __init__(self, metricList=None, bins=None, units=None, metricDtype=float, **kwargs):
        super(MagWrapper,self).__init__(metricList[0].colNameArr.tolist(),
                                          units=metricList[0].units,
                                          metricDtype=metricDtype,
                                          **kwargs)
        import pdb ; pdb.set_trace()
        self.bins = bins
        self.shape = np.size(bins)
        self.metricList = metricList
    def run(self, dataSlice, slicePoint=None):
        result = []
        for metric in self.metricList:
            result.append(metric.run(dataSlice,slicePoint=slicePoint))
        return np.array(result)
        

In [3]:
nside = 16

opsdb = db.OpsimDatabase('minion_1016_sqlite.db')
outDir = 'output'
sql='night < 365'
resultsDb = db.ResultsDb(outDir=outDir)
plotFuncs = [plots.TwoDMap()]

In [ ]:
bundleList = []
mags = np.arange(15.,30,1)
metricList = [metrics.ParallaxMetric(rmag=mag) for mag in mags]
#metric = metrics.ParallaxMetric(rmag=mags)
metric = MagWrapper(metricList=metricList, bins=mags)
slicer = slicers.HealpixSlicer(nside=nside, latCol='ditheredDec', lonCol='ditheredRA')
plotDict = {'xlabel':'Proper Motion Precision'}
bundleList.append(metricBundles.MetricBundle(metric,slicer,sql, plotDict=plotDict, plotFuncs=plotFuncs))

> <ipython-input-2-0380e57c08ae>(9)__init__()
-> self.bins = bins
(Pdb) self.colNameArr
array(['night', 'night'], 
      dtype='|S5')
(Pdb) metricList[0].units
'mas'


In [ ]:
bd = metricBundles.makeBundlesDictFromList(bundleList)
group = metricBundles.MetricBundleGroup(bd, opsdb, outDir=outDir,
                                        resultsDb=resultsDb)
group.runAll()
group.plotAll(closefigs=False)

In [7]:
metric.colNameArr

array(['night', 'night'], 
      dtype='|S5')